<a href="https://colab.research.google.com/github/asjolaviral/scopusapi/blob/main/Scopus_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload query file

Create an excel file **QueryList.xlsx** in following format and upload it.

instName | Query | Year 
-------------------|------------------|------------------
Hon. Shri. Annasaheb Dange College Of Engineering and Technology       | ( AF-ID ( "Hon. Shri. Annasaheb Dange College Of Engineering and Technology"   60076913 )       | 2020-2021
Baba Banda Singh Bahadur Engineering College       | ( AF-ID ( "Baba Banda Singh Bahadur Engineering College" 60097465 )       | 2020-2021
INFLIBNET       | (AF-ID ( "Information and Library Network Centre India" 60004467 )       | 2020-2021

In [ ]:
# Import Libraries
import requests
import json
# import xlwt
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# Import excel file of queries
df = pd.read_excel('QueryList.xlsx')

In [ ]:
df.head()

# Extracting metadata using Scopus API

In [ ]:
# Function to store the data.
def store(i):
  case = {'url':i['prism:url'],
          'title':i['dc:title'],
          'author':i.get('dc:creator','NaN'),
          'publicationname':i.get('prism:publicationName','NaN'),
          'issn':i.get('prism:issn','NaN'),
          'eissn':i.get('prism:eIssn','NaN'),
          'volume':i.get('prism:volume','NaN'),
          'issue':i.get('prism:issueIdentifier','NaN'),
          'pagerange':i.get('prism:pageRange','NaN'),
          'coverdate':i.get('prism:coverDate','NaN'),
          'coverdisplaydate':i.get('prism:coverDisplayDate','NaN'),
          'doi':i.get('prism:doi','NaN'),
          'citedbycount':int(i.get('citedby-count','NaN')),
          'eid':i.get('eid','NaN'),
          'instName':(instName)
          }
  data.append(case)

In [ ]:
# API key and header
key = "xxxxxxxxxx"
headers = {'Accept':'application/json','X-ELS-APIKey':key}

In [ ]:
data = []
# Loop to iterate individual query
for a in df.itertuples():
    instName = a[1]
    instQuery = a[2]
    pubYear = str(a[3])
    Total_record = 0
    start_index = 0
    print('Batch Started for:',instName)
    while Total_record >= start_index:
        url = "https://api.elsevier.com/content/search/scopus?start="+str(start_index)+'&count='+str(25)+'&query='+instQuery+'&date='+pubYear
        response_data = requests.get(url, headers=headers)
        info = response_data.json()
        for i in info['search-results']['entry']:
            store(i)
        print(start_index)
        start_index = start_index + 25
        Total_record = int(info['search-results']['opensearch:totalResults'])
    print(25-(Total_record-start_index),'--Batch Completed for ', instName)

In [ ]:
# Convert case_list to DataFrame and in Excel file
Metadata = pd.DataFrame(data)
Metadata.to_excel('Metadata.xlsx')

# Apply Count and Sum on data group by institute name 
Summary = Metadata.groupby('instName')['citedbycount'].agg(['count','sum'])

# Rename the field name
instPubCitation = Summary.reset_index().rename(columns={'instName':'Institute Name','count':'No of Publications','sum':'Total Citation'})

# Saving final output in Excel file
instPubCitation.to_excel('instPubCitation.xlsx')

# Additional Information

In [ ]:
# Get the quota detail
url = "https://api.elsevier.com/content/search/scopus?query="+instQuery
headers = {'Accept':'application/json','X-ELS-APIKey':key}
r = requests.get(url, headers=headers)
quota = r.headers
print(r.headers)

In [ ]:
import datetime
epochtime = int(quota['X-RateLimit-Reset']) / 1000
datetime = datetime.datetime.fromtimestamp(epochtime)

print('Total Quota',quota['X-RateLimit-Limit'],'\n',
      'Quota Remaining',quota['X-RateLimit-Remaining'],'\n',
      'Quota Reset Date/Time',datetime,'\n',
      'Quota Status',quota['X-ELS-Status']
      )